In [ ]:
# ! pip install pandas geopandas shapely

# Import Packages

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv
from shapely.geometry import box
import matplotlib.pyplot as plt

import sys
sys.path.append("../../src")
from utils.Database import Database

# Set-up Environment

In [ ]:
# load the .env file variables 
load_dotenv()

TABLE_CANADA_DIVISION = os.getenv("TABLE_CANADA_DIVISION")

CACHE_STORAGE_DIR = os.getenv("CACHE_STORAGE_DIR")

ECCC_CACHE_HOURLY_DOWNLOADED_DATA_LIST_FILE = f"{CACHE_STORAGE_DIR}{os.sep}eccc_hourly_downloaded_data_no_dupe_loc.csv"

In [ ]:
REGION_BUFFER_IN_M = 100_000

# Helper Functions

In [ ]:
db = Database()

# Data Loading

In [ ]:
canada_gpd = gpd.read_postgis(
    sql = f"""SELECT * from "{TABLE_CANADA_DIVISION}"; """,
    con = db.connection,
    geom_col = "geometry",
)

In [ ]:
eccc_data_exits_df = pd.read_csv(
    ECCC_CACHE_HOURLY_DOWNLOADED_DATA_LIST_FILE
)
eccc_data_exits_df

In [ ]:
eccc_data_exits_gdf = gpd.GeoDataFrame(
    eccc_data_exits_df,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(
        eccc_data_exits_df['Longitude'], 
        eccc_data_exits_df['Latitude']
    ),
)

In [ ]:
utm_crs = eccc_data_exits_gdf.estimate_utm_crs()

eccc_data_exits_gdf = eccc_data_exits_gdf.to_crs(
    utm_crs
)

In [ ]:
eccc_data_exits_bounds_gdf = gpd.GeoDataFrame()
eccc_data_exits_bounds_gdf['geometry'] = eccc_data_exits_gdf['geometry'].apply(
    lambda point: box(
        point.x - REGION_BUFFER_IN_M,
        point.y - REGION_BUFFER_IN_M,
        point.x + REGION_BUFFER_IN_M,
        point.y + REGION_BUFFER_IN_M
    )
)

# Visualization

In [ ]:
figsize = (20,20)
fig_epsg = 3979

print(f"Plotting Canada...")
ax = canada_gpd.to_crs(
    epsg = fig_epsg
).plot(
    figsize = figsize,
    facecolor = '#FFFFFF00',
    edgecolor = '#888888FF',
)

print(f"Plotting Bounds...")
eccc_data_exits_bounds_gdf.to_crs(
    epsg = fig_epsg,
).plot(
    ax = ax,
    facecolor = '#FFAAAA44',
    edgecolor = '#FF8888AA',
)

print(f"Plotting Stations...")
eccc_data_exits_gdf.to_crs(
    epsg = fig_epsg,
).plot(
    ax = ax,
    marker = "1",
    markersize = 50,
    color = '#FF1111FF',
    label = "Weather Station"
)

plt.legend(
    loc = 'lower left',
    fontsize = 32,
    markerscale = 4
)

plt.axis('off') # no need for axis as it does not showes lat long

plt.savefig(
    f"../../assets/figures/station_rs_cover.png", 
    transparent = True,
    bbox_inches = 'tight', # compress the content  
    pad_inches = 0, # have no extra margin
)

plt.show()

# Find the Non-Overlapping Coverage

In [ ]:
combined_geom = eccc_data_exits_bounds_gdf.unary_union

In [ ]:
combined_gdf = gpd.GeoDataFrame(
    geometry = [combined_geom], 
    crs = eccc_data_exits_bounds_gdf.crs
)

In [ ]:
combined_gdf.to_crs(
    utm_crs
).area / 10**6   #in km^2
# printed value is the non-overlapping area in km^2